# IoT23 Data Preprocessing with 6 million samples 

In [2]:
import os 
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import csv
from numpy import array
from numpy import argmax
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import time as timer

In [3]:
data= pd.read_csv('D:\JupyterNotebook\GitHub\IoT-BotNet-detection-on-IoT23-dataset\IoT23\Data files\iot23_combined_1M.csv',low_memory=False)

In [4]:
data.head()

,Unnamed: 0,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0,1.536227e+09,CeqqKl3hyLQmO8LK98,192.168.100.111,17576.0,78.1.220.212,8081.0,tcp,-,3e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
1,1,1.536227e+09,C2oHQWo1EFGH8D9x7,192.168.100.111,17576.0,152.84.7.111,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
2,2,1.536227e+09,CJLVjs4BByG04mczXc,192.168.100.111,17576.0,173.36.41.67,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
3,3,1.536227e+09,C0z4uS9AWHDH2s4S7,192.168.100.111,17576.0,87.13.21.104,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
4,4,1.536227e+09,CxbNVk3liFNUIlqSPi,192.168.100.111,17576.0,99.110.163.140,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS


Oh..! we have extra column.. lets get rid of it 

In [5]:
del data['Unnamed: 0']

In [5]:
data.shape

(6046623, 21)

Ok...! now we have our data with 21 features ! and around 6 million samples..! 
Its time to clean the data and bring it in good format! 

In [86]:
df= data.drop(columns=['history','id.orig_p','id.resp_p','ts','uid','id.orig_h','id.resp_h','local_orig','local_resp'])

In [87]:
df.replace('-', 0, inplace=True)

In [88]:
df.drop(df[df['label'] == 'C&C-HeartBeat' ].index, inplace = True)

In [89]:
df.drop(df[df['label'] == 'Attack' ].index, inplace = True)

In [90]:
df.drop(df[df['label'] == 'C&C-FileDownload' ].index, inplace = True)

In [91]:
df.drop(df[df['label'] == 'C&C-Mirai' ].index, inplace = True)

In [92]:
df.drop(df[df['label'] == 'C&C-HeartBeat-FileDownload' ].index, inplace = True)

In [93]:
df.drop(df[df['label'] == 'FileDownload' ].index, inplace = True)

In [94]:
df.drop(df[df['label'] == 'C&C-Torii' ].index, inplace = True)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1244131 entries, 0 to 1244675
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   proto          1244131 non-null  object 
 1   service        1244131 non-null  object 
 2   duration       1244131 non-null  object 
 3   orig_bytes     1244131 non-null  object 
 4   resp_bytes     1244131 non-null  object 
 5   conn_state     1244131 non-null  object 
 6   missed_bytes   1244131 non-null  float64
 7   orig_pkts      1244131 non-null  float64
 8   orig_ip_bytes  1244131 non-null  float64
 9   resp_pkts      1244131 non-null  float64
 10  resp_ip_bytes  1244131 non-null  float64
 11  label          1244131 non-null  object 
dtypes: float64(5), object(7)
memory usage: 123.4+ MB


In [96]:
df['orig_bytes']=df['orig_bytes'].astype(float)

In [97]:
df['duration']=df['duration'].astype(float)

In [98]:
df['resp_bytes']=df['resp_bytes'].astype(float)

In [99]:
df.head()

,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,tcp,0,0.000003,0.0,0.0,S0,0.0,2.0,80.0,0.0,0.0,POHS
1,tcp,0,0.000002,0.0,0.0,S0,0.0,2.0,80.0,0.0,0.0,POHS
2,tcp,0,0.000002,0.0,0.0,S0,0.0,2.0,80.0,0.0,0.0,POHS
3,tcp,0,0.000002,0.0,0.0,S0,0.0,2.0,80.0,0.0,0.0,POHS
4,tcp,0,0.000002,0.0,0.0,S0,0.0,2.0,80.0,0.0,0.0,POHS


In [100]:
X=df.iloc[:,:-1]

In [101]:
y=df['label']

In [102]:
cat_df =X.select_dtypes(include=['object']).copy()

In [103]:
cat_df

,proto,service,conn_state
0,tcp,0,S0
1,tcp,0,S0
2,tcp,0,S0
3,tcp,0,S0
4,tcp,0,S0
...,...,...,...
1244671,tcp,0,S0
1244672,tcp,0,S0
1244673,tcp,0,S0
1244674,tcp,0,S0


In [104]:
cat_new= pd.get_dummies(cat_df, columns=['proto','service','conn_state'])

In [105]:
cat_new

,proto_icmp,proto_tcp,proto_udp,service_0,service_dhcp,service_dns,service_http,service_irc,service_ssl,conn_state_OTH,...,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244671,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1244672,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1244673,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1244674,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [106]:
num_df =X.select_dtypes(exclude=['object']).copy()

In [107]:
Xe = pd.concat([num_df,cat_new], axis=1)

In [108]:
Xe.head()

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,proto_icmp,proto_tcp,...,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.000003,0.0,0.0,0.0,2.0,80.0,0.0,0.0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,0.000002,0.0,0.0,0.0,2.0,80.0,0.0,0.0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,0.000002,0.0,0.0,0.0,2.0,80.0,0.0,0.0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,0.000002,0.0,0.0,0.0,2.0,80.0,0.0,0.0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0.000002,0.0,0.0,0.0,2.0,80.0,0.0,0.0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [109]:
Xe = np.asarray(Xe).astype(np.float32)

In [110]:
le=LabelEncoder()

In [111]:
Y=le.fit_transform(y)

In [112]:
Y

array([4, 4, 4, ..., 4, 4, 4])

In [113]:
x_scaler=MinMaxScaler()

In [114]:
x_scaler.fit(Xe)

MinMaxScaler()

In [115]:
Normalized_x=x_scaler.transform(Xe)

In [116]:
Normalized_x

array([[6.1253468e-11, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.0835644e-11, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.0835644e-11, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [30]:
Normalized_x.shape

(1244676, 29)

In [141]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import auc,roc_auc_score,roc_curve,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [118]:
X_train, X_test, Y_train, Y_test = train_test_split(Xe, Y, random_state=10, test_size=0.20)

In [119]:
X_train.shape

(995304, 29)

In [120]:
X_test.shape

(248827, 29)

In [59]:
GuassianNB?

Object `GuassianNB` not found.


In [35]:
start = timer.time()
gnb_model = GaussianNB()
gnb_model.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 0.52 seconds


In [36]:
# Predicting the test set results
y_gnb = gnb_model.predict(X_test)
y_gnb_prob = gnb_model.predict_proba(X_test)

In [37]:
print("Classification report for GNB: \n{}".format(classification_report(Y_test,y_gnb)))
print("Confusion matrix for GNB: \n{}".format(confusion_matrix(Y_test,y_gnb)))
print("Accuracy score for GNB: {:.2f}".format(accuracy_score(Y_test,y_gnb)))
# calculate precision, recall, and f1 scores
prec_gnb = precision_score(Y_test,y_gnb,average='weighted',zero_division=1)
rec_gnb = recall_score(Y_test,y_gnb,average='weighted',zero_division=1)
f1_gnb = f1_score(Y_test,y_gnb,average='weighted',zero_division=1)
print("Precision score for GNB: {:.2f}".format(prec_gnb))
print("Recall score for GNB: {:.2f}".format(rec_gnb))
print("F1 score for GNB: {:.2f}".format(f1_gnb))

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report for GNB: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       1.00      0.00      0.00     29826
           2       0.56      0.12      0.19      2960
           3       0.67      0.86      0.75         7
           4       1.00      0.14      0.25        57
           5       1.00      0.75      0.86         4
           6       0.00      0.00      0.00         1
           7       0.33      0.20      0.25         5
           8       0.57      0.00      0.00     27658
           9       0.50      0.67      0.57         3
          10       0.21      1.00      0.35     52797
          11       0.85      0.00      0.00    135596

    accuracy                           0.21    248936
   macro avg       0.56      0.31      0.27    248936
weighted avg       0.70      0.21      0.08    248936

Confusion matrix for GNB: 
[[     0      0      1      0      0      0      0      0      0      0
  

In [146]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [147]:
train_scores, test_scores = list(), list()
values = [i for i in range(1, 20)]

In [148]:
cv = KFold(n_splits=10, random_state=21, shuffle=True)

In [124]:
for i in values:
	# configure the model
	model = DecisionTreeClassifier(max_depth=i)
	# fit model on the training dataset
	model.fit(X_train, Y_train)
	# evaluate on the train dataset
	train_yhat = model.predict(X_train)
	train_acc = accuracy_score(Y_train, train_yhat)
	train_scores.append(train_acc)
	# evaluate on the test dataset
	test_yhat = model.predict(X_test)
	test_acc = accuracy_score(Y_test, test_yhat)
	test_scores.append(test_acc)
	# summarize progress
	print('>%d, train: %.4f, test: %.4f' % (i, train_acc, test_acc))

>1, train: 0.6363, test: 0.6355
>2, train: 0.6363, test: 0.6355
>3, train: 0.6873, test: 0.6863
>4, train: 0.6902, test: 0.6886
>5, train: 0.7294, test: 0.7279
>6, train: 0.7309, test: 0.7293
>7, train: 0.7320, test: 0.7305
>8, train: 0.7321, test: 0.7306
>9, train: 0.7321, test: 0.7306
>10, train: 0.7321, test: 0.7306
>11, train: 0.7321, test: 0.7306
>12, train: 0.7321, test: 0.7306
>13, train: 0.7322, test: 0.7306
>14, train: 0.7322, test: 0.7305
>15, train: 0.7322, test: 0.7305
>16, train: 0.7322, test: 0.7305
>17, train: 0.7322, test: 0.7304
>18, train: 0.7323, test: 0.7304
>19, train: 0.7323, test: 0.7304


Trying to investigate Balanced Data 

In [133]:
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [126]:
print("Total class values: ", Counter(y))

Total class values:  Counter({'POHS': 677827, 'Okiru': 262690, 'Benign': 149744, 'DDoS': 138777, 'C&C': 15093})


In [127]:
%%time
from imblearn.over_sampling import SMOTE

Wall time: 0 ns


In [129]:
from imblearn.over_sampling import RandomOverSampler 

In [130]:
ros = RandomOverSampler(random_state=42)

In [131]:
X_res, y_res = ros.fit_resample(X_train, Y_train)

In [135]:
counter = Counter(Y_train)

In [136]:
for k,v in counter.items():
	per = v / len(Y_train) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=4, n=542401 (54.496%)
Class=2, n=111112 (11.164%)
Class=3, n=209781 (21.077%)
Class=0, n=119944 (12.051%)
Class=1, n=12066 (1.212%)


In [139]:
Counter(Y_train)

Counter({4: 542401, 2: 111112, 3: 209781, 0: 119944, 1: 12066})

In [140]:
Counter(y_res)

Counter({4: 542401, 2: 542401, 3: 542401, 0: 542401, 1: 542401})

In [142]:
X_train, X_test, Y_train, Y_test = train_test_split(X_res, y_res, random_state=10, test_size=0.20)

In [143]:
start = timer.time()
gnb_model = GaussianNB()
gnb_model.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 0.95 seconds


In [144]:
# Predicting the test set results
y_gnb = gnb_model.predict(X_test)
y_gnb_prob = gnb_model.predict_proba(X_test)

In [145]:
print("Classification report for GNB: \n{}".format(classification_report(Y_test,y_gnb)))
print("Confusion matrix for GNB: \n{}".format(confusion_matrix(Y_test,y_gnb)))
print("Accuracy score for GNB: {:.2f}".format(accuracy_score(Y_test,y_gnb)))
# calculate precision, recall, and f1 scores
prec_gnb = precision_score(Y_test,y_gnb,average='weighted',zero_division=1)
rec_gnb = recall_score(Y_test,y_gnb,average='weighted',zero_division=1)
f1_gnb = f1_score(Y_test,y_gnb,average='weighted',zero_division=1)
print("Precision score for GNB: {:.2f}".format(prec_gnb))
print("Recall score for GNB: {:.2f}".format(rec_gnb))
print("F1 score for GNB: {:.2f}".format(f1_gnb))

Classification report for GNB: 
              precision    recall  f1-score   support

           0       0.76      0.00      0.00    108814
           1       0.92      0.11      0.20    108178
           2       0.70      0.00      0.00    108756
           3       0.21      1.00      0.34    108170
           4       0.97      0.08      0.15    108483

    accuracy                           0.24    542401
   macro avg       0.71      0.24      0.14    542401
weighted avg       0.71      0.24      0.14    542401

Confusion matrix for GNB: 
[[    53    997      5 107541    218]
 [    14  12081      0  96015     68]
 [     2      0     21 108731      2]
 [     0      0      0 108170      0]
 [     1      0      4  99831   8647]]
Accuracy score for GNB: 0.24
Precision score for GNB: 0.71
Recall score for GNB: 0.24
F1 score for GNB: 0.14


In [149]:
for i in values:
	# configure the model
	model = DecisionTreeClassifier(max_depth=i)
	# fit model on the training dataset
	model.fit(X_train, Y_train)
	# evaluate on the train dataset
	train_yhat = model.predict(X_train)
	train_acc = accuracy_score(Y_train, train_yhat)
	train_scores.append(train_acc)
	# evaluate on the test dataset
	test_yhat = model.predict(X_test)
	test_acc = accuracy_score(Y_test, test_yhat)
	test_scores.append(test_acc)
	# summarize progress
	print('>%d, train: %.4f, test: %.4f' % (i, train_acc, test_acc))

>1, train: 0.3639, test: 0.3638
>2, train: 0.5450, test: 0.5445
>3, train: 0.5633, test: 0.5631
>4, train: 0.6503, test: 0.6495
>5, train: 0.6578, test: 0.6578
>6, train: 0.6726, test: 0.6729
>7, train: 0.6773, test: 0.6777
>8, train: 0.6785, test: 0.6789
>9, train: 0.6790, test: 0.6794
>10, train: 0.6794, test: 0.6797
>11, train: 0.6796, test: 0.6799
>12, train: 0.6806, test: 0.6807
>13, train: 0.6809, test: 0.6811
>14, train: 0.6814, test: 0.6813
>15, train: 0.6826, test: 0.6826
>16, train: 0.6836, test: 0.6837
>17, train: 0.6843, test: 0.6844
>18, train: 0.6853, test: 0.6853
>19, train: 0.6864, test: 0.6863
